In [18]:
from google.colab import files
from google.colab import drive
import os
import io
import pandas as pd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import chisquare
import numpy as np

from datetime import datetime

In [19]:
drive.mount('/content/drive') #/MyDrive/Colab Notebooks/데이터/기상데이터

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
lon_lat = pd.read_csv('drive/MyDrive/mapping_0713.csv', index_col = 0)

# Cloud data

In [21]:
rn = pd.read_csv('drive/MyDrive/rn.csv', index_col = 0)
ta = pd.read_csv('drive/MyDrive/ta.csv', index_col = 0)
wind = pd.read_csv('drive/MyDrive/db_aws_wind_dd.csv', index_col = 0)
cloud = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/db_cloud.csv', index_col = 0)

In [22]:
# cloud, flight, ev, rn, rhm, prsr, ta, wind
cloud = cloud.rename(columns={'a.tm' : 'tma', 'a.stn_id' : 'stn_id', 'a.clfm_abbr_cd' : 'cloud.clfm_abbr_cd', 'a.ca' : 'cloud.ca', 'a.ch' : 'cloud.ch'})
rn = rn.rename(columns = {'a.tma' : 'tma', 'a.stn_id' : 'stn_id', 'a.sum_rn' : 'rn.sum_rn', 'a.hr1_max_rn' : 'rn.hr1_max_rn'})
ta = ta.rename(columns = {'a.tma' : 'tma', 'a.stn_id' : 'stn_id', 'a.avg_ta' : 'ta.avg_ta', 'a.max_ta' : 'ta.max_ta', 'a.min_ta' : 'ta.min_ta'})
wind = wind.rename(columns = {'a.tma' : 'tma', 'a.stn_id' : 'stn_id', 'a.avg_ws' : 'wind.avg_ws', 'a.mix_ws' : 'wind.mix_ws', 'a.max_ws' : 'wind.max_ws', 'a.max_wd' : 'wind.max_wd', 'a.max_ws_wd' : 'wind.max_ws_wd', 'a.max_ins_ws' : 'wind.max_ins_ws', 'a.max_ins_ws_wd' : 'wind_max_ins_ws_wd', 'a.mix_wd' : 'wind.mix_wd'})

In [23]:
cloud

,tma,stn_id,cloud.clfm_abbr_cd,cloud.ca,cloud.ch
X,,,,,
1,2012-02-01 00:00:00.0,100,Sc,5.0,2.0
2,2012-02-01 00:00:00.0,112,Sc,3.0,10.0
3,2012-02-01 00:00:00.0,115,Cu,3.0,7.0
4,2012-02-01 00:00:00.0,119,Sc,4.0,10.0
5,2012-02-01 00:00:00.0,130,As,10.0,30.0
...,...,...,...,...,...
1953829,2022-10-30 08:00:00.0,104,St,6.0,6.0
1953830,2022-10-30 08:00:00.0,131,Sc,5.0,10.0
1953831,2022-10-30 08:00:00.0,143,Sc,5.0,10.0


In [16]:
cloud = pd.merge(cloud, lon_lat, on = ['stn_id'])
rn = pd.merge(rn, lon_lat, on = ['stn_id'])
ta = pd.merge(ta, lon_lat, on = ['stn_id'])
wind = pd.merge(wind, lon_lat, on = ['stn_id'])

In [17]:
cloud

,tma,stn_id,cloud.clfm_abbr_cd,cloud.ca,cloud.ch,name,address,lat,long,add
0,2012-02-01 00:00:00.0,100,Sc,5.0,2.0,대관령,강원도 평창군 대관령면 횡계리,37.677115,128.71832,강원
1,2012-02-03 21:00:00.0,100,Ci,3.0,55.0,대관령,강원도 평창군 대관령면 횡계리,37.677115,128.71832,강원
2,2012-02-04 01:00:00.0,100,Sc,7.0,2.0,대관령,강원도 평창군 대관령면 횡계리,37.677115,128.71832,강원
3,2012-02-04 02:00:00.0,100,Sc,7.0,2.0,대관령,강원도 평창군 대관령면 횡계리,37.677115,128.71832,강원
4,2012-02-04 03:00:00.0,100,Sc,8.0,2.0,대관령,강원도 평창군 대관령면 횡계리,37.677115,128.71832,강원
...,...,...,...,...,...,...,...,...,...,...
1953564,2022-10-29 12:00:00.0,177,Ci,1.0,60.0,홍성,충청남도 홍성군 홍북면 신경리 51-2,36.657545,126.68771,충남
1953565,2022-10-29 18:00:00.0,177,Ci,7.0,60.0,홍성,충청남도 홍성군 홍북면 신경리 51-2,36.657545,126.68771,충남
1953566,2022-10-30 07:00:00.0,177,St,9.0,3.0,홍성,충청남도 홍성군 홍북면 신경리 51-2,36.657545,126.68771,충남
1953567,2022-10-30 08:00:00.0,177,St,5.0,1.0,홍성,충청남도 홍성군 홍북면 신경리 51-2,36.657545,126.68771,충남


In [ ]:
cloud['date'] = cloud['tma'].map(lambda date_cloud: date_cloud[:10])
cloud['time'] = cloud['tma'].map(lambda time_cloud: time_cloud[11:-2])
cloud['date'] = pd.to_datetime(cloud['date'], format='%Y-%m-%d')
cloud['time'] = pd.to_datetime(cloud['time'], format='%H:%M:%S')

rn['date'] = rn['tma'].map(lambda date_rn: date_rn[:10])
rn['time'] = rn['tma'].map(lambda time_rn: time_rn[11:-2])
rn['date'] = pd.to_datetime(rn['date'], format='%Y-%m-%d')
rn['time'] = pd.to_datetime(rn['time'], format='%H:%M:%S')

ta['date'] = ta['tma'].map(lambda date_ta: date_ta[:10])
ta['time'] = ta['tma'].map(lambda time_ta: time_ta[11:-2])
ta['date'] = pd.to_datetime(ta['date'], format='%Y-%m-%d')
ta['time'] = pd.to_datetime(ta['time'], format='%H:%M:%S')

wind['date'] = wind['tma'].map(lambda date_wind: date_wind[:10])
wind['time'] = wind['tma'].map(lambda time_wind: time_wind[11:-2])
wind['date'] = pd.to_datetime(wind['date'], format='%Y-%m-%d')
wind['time'] = pd.to_datetime(wind['time'], format='%H:%M:%S')

In [ ]:
## cloud, flight, rn, rhm, prsr, ta, wind
del cloud['name']
del cloud['address']

del rn['name']
del rn['address']

del ta['name']
del ta['address']

del wind['name']
del wind['address']


In [ ]:
cloud_ds = cloud.groupby(['date', 'stn_id'], as_index=False).median()
rn_ds = rn.groupby(['date', 'stn_id'], as_index=False).median()
ta_ds = ta.groupby(['date', 'stn_id'], as_index=False).median()
wind_ds = wind.groupby(['date', 'stn_id'], as_index=False).median()

In [ ]:
rn_ds = pd.merge(rn_ds, lon_lat, on = ['stn_id'])
ta_ds = pd.merge(ta_ds, lon_lat, on = ['stn_id'])

In [ ]:
del rn_ds['name']
del rn_ds['address']
del rn_ds['lat_x']
del rn_ds['long_y']

del ta_ds['name']
del ta_ds['address']
del ta_ds['lat_x']
del ta_ds['long_y']

In [ ]:
rn_ds_m = rn_ds.groupby(['date', 'add'], as_index=False).median()
ta_ds_m = ta_ds.groupby(['date', 'add'], as_index=False).median()

In [ ]:
rn_ds

,date,stn_id,rn.sum_rn,rn.hr1_max_rn,lat_x,long_x,name,address,lat_y,long_y,add
0,2012-02-01,12,0.0,0.0,36.53330,126.31670,안면도(감),충남,36.53330,126.31670,충남
1,2012-02-02,12,0.0,0.0,36.53330,126.31670,안면도(감),충남,36.53330,126.31670,충남
2,2012-02-03,12,0.0,0.0,36.53330,126.31670,안면도(감),충남,36.53330,126.31670,충남
3,2012-02-04,12,0.0,0.0,36.53330,126.31670,안면도(감),충남,36.53330,126.31670,충남
4,2012-02-05,12,0.0,0.0,36.53330,126.31670,안면도(감),충남,36.53330,126.31670,충남
...,...,...,...,...,...,...,...,...,...,...,...
2193072,2022-10-24,239,0.0,0.0,36.48522,127.24438,세종,충청남도 금산군 금산읍 아인리 134-5,36.48522,127.24438,충남
2193073,2022-10-25,239,0.0,0.0,36.48522,127.24438,세종,충청남도 금산군 금산읍 아인리 134-5,36.48522,127.24438,충남
2193074,2022-10-26,239,0.0,0.0,36.48522,127.24438,세종,충청남도 금산군 금산읍 아인리 134-5,36.48522,127.24438,충남
2193075,2022-10-27,239,0.0,0.0,36.48522,127.24438,세종,충청남도 금산군 금산읍 아인리 134-5,36.48522,127.24438,충남


In [ ]:
rn_ds_m.isna().sum()
ta_ds_m.isna().sum()

date         0
add          0
stn_id       0
ta.avg_ta    0
ta.max_ta    0
ta.min_ta    0
long_x       0
lat_y        0
dtype: int64

In [ ]:
set(rn_ds_m['add'])
set(ta_ds_m['add'])

{'강원',
 '경기',
 '경남',
 '경북',
 '광주',
 '대구',
 '대전',
 '부산',
 '서울',
 '세종',
 '울산',
 '인천',
 '전남',
 '전북',
 '제주',
 '충남',
 '충북'}

In [ ]:
rn_ds_m.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/rn_median.csv')
ta_ds_m.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/ta_median.csv')

In [ ]:
# cloud, wind, rn, ta

cloud = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/cloud_median.csv')
wind = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/wind_median.csv')
ta = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/ta_median.csv')
rn = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/rn_median.csv')

In [ ]:
cloud_wind = pd.merge(cloud, wind, on = ['date', 'add'])
ta_rn = pd.merge(ta, rn, on = ['date', 'add'])
all_merged = pd.merge(cloud_wind, ta_rn, on = ['date', 'add'])

In [ ]:
#del all_merged['Unnamed: 0_x_x']
#del all_merged['stn_id_x_x']
del all_merged['lat_x']
del all_merged['long_x']
del all_merged['Unnamed: 0_y_x']
del all_merged['stn_id_y_x']
del all_merged['lat_y']
del all_merged['long_y']
del all_merged['Unnamed: 0_x_y']
del all_merged['long_x_x']
del all_merged['lat_y_x']
del all_merged['Unnamed: 0_y_y']
del all_merged['stn_id_y_y']
del all_merged['long_x_y']
del all_merged['lat_y_y']



In [ ]:
del all_merged['stn_id_x_y']

In [ ]:
all_merged.columns

Index(['date', 'add', 'cloud.ca', 'cloud.ch', 'wind.avg_ws', 'wind.mix_ws',
       'wind.max_ws', 'wind.max_wd', 'wind.max_ws_wd', 'wind.max_ins_ws',
       'wind_max_ins_ws_wd', 'wind.mix_wd', 'stn_id_x_y', 'ta.avg_ta',
       'ta.max_ta', 'ta.min_ta', 'rn.sum_rn', 'rn.hr1_max_rn'],
      dtype='object')

'date', 'add', 'cloud.ca', 'cloud.ch', 'wind.avg_ws', 'wind.mix_ws',
       'wind.max_ws', 'wind.max_wd', 'wind.max_ws_wd', 'wind.max_ins_ws',
       'wind_max_ins_ws_wd', 'wind.mix_wd', 'ta.avg_ta', 'ta.max_ta',
       'ta.min_ta', 'rn.sum_rn', 'rn.hr1_max_rn'

In [ ]:
all_merged.isna().sum()

date                  0
add                   0
cloud.ca              0
cloud.ch              0
wind.avg_ws           0
wind.mix_ws           0
wind.max_ws           0
wind.max_wd           0
wind.max_ws_wd        0
wind.max_ins_ws       0
wind_max_ins_ws_wd    0
wind.mix_wd           0
ta.avg_ta             0
ta.max_ta             0
ta.min_ta             0
rn.sum_rn             0
rn.hr1_max_rn         0
dtype: int64

In [ ]:
all_merged = all_merged.rename(columns={'cloud.ca' : 'ca', 'cloud.ch' : 'ch', 'wind.avg_ws' : 'avg_ws',
                              'wind.mix_ws' : 'mix_ws', 'wind.max_ws' : 'max_ws', 'wind.max_wd' : 'max_wd',
                              'wind.max_wd' : 'max_wd', 'wind.max_ws_wd' : 'max_ws_wd', 
                              'wind.max_ins_ws' : 'max_ins_ws', 'wind_max_ins_ws_wd' : 'max_ins_ws_wd',
                              'wind.mix_wd' : 'mix_wd', 'ta.avg_ta' : 'avg.ta', 'ta.max_ta' : 'max_ta', 
                              'ta.min_ta' : 'min_ta', 'rn.sum_rn' : 'sum_rn', 'rn.hr1_max_rn' : 'hr1_max_rn'})

In [ ]:
all_merged.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/merged_median.csv')

In [ ]:
rain = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/호우요약최종.csv')
merged_fixing = pd.merge(all_merged, rain, on = ['date', 'add'], how='outer')
merged_fixing['typhoon_yn'][merged_fixing['typhoon_yn'].isna() == True] = 0
merged_fixing['damage_conv'][merged_fixing['damage_conv'].isna() == True] = 0
merged_fixing['rain_yn'][merged_fixing['rain_yn'].isna() == True] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [ ]:
merged_fixing.isna().sum()

date             0
add              0
ca               0
ch               0
avg_ws           0
mix_ws           0
max_ws           0
max_wd           0
max_ws_wd        0
max_ins_ws       0
max_ins_ws_wd    0
mix_wd           0
avg.ta           0
max_ta           0
min_ta           0
sum_rn           0
hr1_max_rn       0
typhoon_yn       0
damage_conv      0
rain_yn          0
dtype: int64

In [ ]:
del merged_fixing['Unnamed: 0']

In [ ]:
merged_fixing.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/기상호우태풍병합데이터(version2).csv')

In [ ]:
merged_final = merged_fixing
merged_final['rain_yn'] = merged_final['rain_yn'].astype(int)
print(type(merged_final['typhoon_yn'][0]))  # typhoon_yn	rain_y
print(type(merged_final['rain_yn'][0]))

<class 'numpy.float64'>
<class 'numpy.int64'>


In [ ]:
df_with_notyphoon = merged_final[merged_final['typhoon_yn']==0]
df_with_rain = merged_final[merged_final['rain_yn']==1]

del df_with_notyphoon['typhoon_yn']
df_notyphoon_norain = df_with_notyphoon[df_with_notyphoon['rain_yn']==0]
del df_notyphoon_norain['rain_yn']
df_notyphoon_norain # 호우, 태풍 둘 다 없는 날

df_notyphoon_yesrain = df_with_notyphoon[df_with_notyphoon['rain_yn']==1]
del df_notyphoon_yesrain['rain_yn']
df_notyphoon_yesrain # 호우는 내렸지만 태풍은 없던 날

df_yesrain_yestyphoon = df_with_rain[df_with_rain['typhoon_yn'] == 1]
del df_yesrain_yestyphoon['typhoon_yn']
del df_yesrain_yestyphoon['rain_yn']
df_yesrain_yestyphoon   # 호우랑 태풍 둘다 있었던 날. 정확하게는 태풍이 있어서 호우가 있었던 날

df_notyphoon_norain.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/df_notyphoon_norain_median.csv')
df_notyphoon_yesrain.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/df_notyphoon_yesrain_median.csv')
df_yesrain_yestyphoon.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/df_yesrain_yestyphoon_median.csv')



In [ ]:
df_yesrain_yestyphoon.isna().sum()

date             0
add              0
ca               0
ch               0
avg_ws           0
mix_ws           0
max_ws           0
max_wd           0
max_ws_wd        0
max_ins_ws       0
max_ins_ws_wd    0
mix_wd           0
avg.ta           0
max_ta           0
min_ta           0
sum_rn           0
hr1_max_rn       0
damage_conv      0
dtype: int64

------------------------------------------------------------------------------------------------------------------------

In [4]:
data = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/기상호우태풍병합데이터(version2).csv')
data

,Unnamed: 0,date,add,ca,ch,avg_ws,mix_ws,max_ws,max_wd,max_ws_wd,...,max_ins_ws_wd,mix_wd,avg.ta,max_ta,min_ta,sum_rn,hr1_max_rn,typhoon_yn,damage_conv,rain_yn
0,0,2012-02-01,강원,7.000000,10.000000,2.45,2.15,5.60,247.50,259.50,...,264.35,255.20,-12.90,-5.85,-18.80,0.0,0.0,0.0,0.0,0.0
1,1,2012-02-01,경기,3.500000,10.000000,2.15,1.80,4.80,270.00,276.25,...,278.70,278.15,-12.00,-6.40,-16.35,0.0,0.0,0.0,0.0,0.0
2,2,2012-02-01,경남,6.000000,10.000000,4.00,3.50,6.90,292.50,281.80,...,288.25,299.40,-1.70,4.80,-7.50,0.0,0.0,0.0,0.0,0.0
3,3,2012-02-01,경북,6.000000,8.000000,3.40,3.10,6.30,292.50,283.30,...,292.50,290.10,-6.90,-0.10,-11.30,0.5,0.5,0.0,0.0,0.0
4,4,2012-02-01,광주,6.000000,10.000000,3.35,3.40,6.30,326.25,312.65,...,308.00,329.95,-4.05,1.70,-6.55,0.0,0.5,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56253,56253,2022-10-24,세종,2.333333,22.333333,1.50,1.20,4.80,292.50,138.85,...,301.80,310.25,8.60,15.75,3.05,0.0,0.0,0.0,0.0,0.0
56254,56254,2022-10-25,세종,2.000000,10.000000,0.85,0.55,3.00,56.25,56.45,...,58.50,47.45,7.90,17.05,0.20,0.0,0.0,0.0,0.0,0.0
56255,56255,2022-10-26,세종,7.500000,60.000000,0.65,0.30,2.20,67.50,98.30,...,121.75,78.60,9.75,18.75,1.75,0.0,0.0,0.0,0.0,0.0
56256,56256,2022-10-27,세종,5.666667,57.666667,0.80,0.55,3.45,191.25,300.35,...,267.55,181.65,11.30,20.30,3.80,0.0,0.0,0.0,0.0,0.0


In [5]:
set(data['add'])

{'강원',
 '경기',
 '경남',
 '경북',
 '광주',
 '대구',
 '대전',
 '부산',
 '서울',
 '세종',
 '울산',
 '인천',
 '전남',
 '전북',
 '제주',
 '충남',
 '충북'}

In [9]:
print(len(data[data['add'] == '강원']))
print(len(data[data['add'] == '경기']))
print(len(data[data['add'] == '경남']))
print(len(data[data['add'] == '경북']))
print(len(data[data['add'] == '광주']))
print(len(data[data['add'] == '대구']))
print(len(data[data['add'] == '대전']))
print(len(data[data['add'] == '부산']))
print(len(data[data['add'] == '서울']))
print(len(data[data['add'] == '세종']))
print(len(data[data['add'] == '울산']))
print(len(data[data['add'] == '인천']))
print(len(data[data['add'] == '전남']))
print(len(data[data['add'] == '전북']))
print(len(data[data['add'] == '제주']))
print(len(data[data['add'] == '충남']))
print(len(data[data['add'] == '충북']))


3412
3695
3368
3413
3211
3187
3202
3234
3250
3241
3179
3373
3506
3243
3263
3268
3213


In [34]:
cloud['date'] = cloud['tma'].map(lambda date_cloud: date_cloud[:10])
cloud['time'] = cloud['tma'].map(lambda time_cloud: time_cloud[11:-2])
cloud['date'] = pd.to_datetime(cloud['date'], format='%Y-%m-%d')
cloud['time'] = pd.to_datetime(cloud['time'], format='%H:%M:%S')

In [33]:
len(set(cloud['date']))

3305

In [30]:
days = pd.date_range(start="2012-02-01",end="2022-10-31")
len(days)

3926

In [41]:
days.difference(cloud.date)

DatetimeIndex(['2012-12-01', '2012-12-02', '2012-12-03', '2012-12-04',
               '2012-12-05', '2012-12-06', '2012-12-07', '2012-12-08',
               '2012-12-09', '2012-12-10',
               ...
               '2022-01-23', '2022-01-24', '2022-01-25', '2022-01-26',
               '2022-01-27', '2022-01-28', '2022-01-29', '2022-01-30',
               '2022-01-31', '2022-10-31'],
              dtype='datetime64[ns]', length=621, freq=None)

In [50]:
data[data['date']=='2012-02-05']

,Unnamed: 0,date,add,ca,ch,avg_ws,mix_ws,max_ws,max_wd,max_ws_wd,...,max_ins_ws_wd,mix_wd,avg.ta,max_ta,min_ta,sum_rn,hr1_max_rn,typhoon_yn,damage_conv,rain_yn
56,56,2012-02-05,강원,7.0,60.000000,1.10,0.55,3.25,202.50,223.60,...,213.70,215.55,-4.90,3.90,-10.25,0.0,0.0,0.0,0.0,0.0
57,57,2012-02-05,경기,6.0,60.000000,0.60,0.30,2.20,90.00,149.25,...,146.90,105.90,-2.90,4.80,-7.10,0.0,0.0,0.0,0.0,0.0
58,58,2012-02-05,경남,8.0,28.000000,0.90,0.30,2.70,202.50,191.30,...,215.15,268.50,-0.85,5.50,-6.10,0.0,0.0,0.0,0.0,0.0
59,59,2012-02-05,경북,6.0,60.000000,0.90,0.40,3.10,225.00,237.10,...,244.65,254.70,-1.90,6.50,-7.75,0.0,0.0,0.0,0.0,0.0
60,60,2012-02-05,광주,5.5,28.000000,1.05,0.65,2.90,213.75,110.55,...,119.55,228.00,0.30,6.35,-4.50,0.0,0.0,0.0,0.0,0.0
61,61,2012-02-05,대구,7.0,30.000000,1.05,0.40,2.80,258.75,275.85,...,284.10,306.95,0.25,7.15,-4.65,0.0,0.0,0.0,0.0,0.0
62,62,2012-02-05,대전,6.0,33.000000,0.50,0.40,1.85,247.50,303.50,...,285.45,342.80,-1.65,5.35,-6.05,0.0,0.0,0.0,0.0,0.0
63,63,2012-02-05,부산,9.0,28.000000,1.75,1.35,3.90,281.25,290.95,...,294.05,303.90,2.85,6.00,-0.15,0.0,0.0,0.0,0.0,0.0
64,64,2012-02-05,서울,5.5,60.000000,1.00,0.50,2.40,67.50,83.90,...,69.20,44.00,-0.90,4.20,-4.50,0.0,0.0,0.0,0.0,0.0
65,65,2012-02-05,울산,8.0,28.000000,2.40,1.30,4.70,247.50,205.30,...,244.70,316.20,2.55,7.80,-2.40,0.0,0.0,0.0,0.0,0.0


In [45]:
data

,Unnamed: 0,date,add,ca,ch,avg_ws,mix_ws,max_ws,max_wd,max_ws_wd,...,max_ins_ws_wd,mix_wd,avg.ta,max_ta,min_ta,sum_rn,hr1_max_rn,typhoon_yn,damage_conv,rain_yn
0,0,2012-02-01,강원,7.000000,10.000000,2.45,2.15,5.60,247.50,259.50,...,264.35,255.20,-12.90,-5.85,-18.80,0.0,0.0,0.0,0.0,0.0
1,1,2012-02-01,경기,3.500000,10.000000,2.15,1.80,4.80,270.00,276.25,...,278.70,278.15,-12.00,-6.40,-16.35,0.0,0.0,0.0,0.0,0.0
2,2,2012-02-01,경남,6.000000,10.000000,4.00,3.50,6.90,292.50,281.80,...,288.25,299.40,-1.70,4.80,-7.50,0.0,0.0,0.0,0.0,0.0
3,3,2012-02-01,경북,6.000000,8.000000,3.40,3.10,6.30,292.50,283.30,...,292.50,290.10,-6.90,-0.10,-11.30,0.5,0.5,0.0,0.0,0.0
4,4,2012-02-01,광주,6.000000,10.000000,3.35,3.40,6.30,326.25,312.65,...,308.00,329.95,-4.05,1.70,-6.55,0.0,0.5,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56253,56253,2022-10-24,세종,2.333333,22.333333,1.50,1.20,4.80,292.50,138.85,...,301.80,310.25,8.60,15.75,3.05,0.0,0.0,0.0,0.0,0.0
56254,56254,2022-10-25,세종,2.000000,10.000000,0.85,0.55,3.00,56.25,56.45,...,58.50,47.45,7.90,17.05,0.20,0.0,0.0,0.0,0.0,0.0
56255,56255,2022-10-26,세종,7.500000,60.000000,0.65,0.30,2.20,67.50,98.30,...,121.75,78.60,9.75,18.75,1.75,0.0,0.0,0.0,0.0,0.0
56256,56256,2022-10-27,세종,5.666667,57.666667,0.80,0.55,3.45,191.25,300.35,...,267.55,181.65,11.30,20.30,3.80,0.0,0.0,0.0,0.0,0.0


In [53]:
data[data['add']=='서울']
data[data['add']=='강원']

,Unnamed: 0,date,add,ca,ch,avg_ws,mix_ws,max_ws,max_wd,max_ws_wd,...,max_ins_ws_wd,mix_wd,avg.ta,max_ta,min_ta,sum_rn,hr1_max_rn,typhoon_yn,damage_conv,rain_yn
0,0,2012-02-01,강원,7.0,10.0,2.45,2.15,5.60,247.5,259.50,...,264.35,255.20,-12.90,-5.85,-18.80,0.0,0.0,0.0,0.0,0.0
26,26,2012-02-03,강원,4.0,30.0,2.40,1.80,5.60,225.0,242.90,...,239.10,241.30,-12.95,-5.45,-21.85,0.0,0.0,0.0,0.0,0.0
40,40,2012-02-04,강원,7.0,9.0,2.40,1.50,5.80,247.5,251.00,...,248.90,253.05,-6.20,0.20,-11.45,0.0,0.0,0.0,0.0,0.0
56,56,2012-02-05,강원,7.0,60.0,1.10,0.55,3.25,202.5,223.60,...,213.70,215.55,-4.90,3.90,-10.25,0.0,0.0,0.0,0.0,0.0
72,72,2012-02-06,강원,6.0,60.0,1.05,0.50,3.80,225.0,224.85,...,236.25,239.25,-2.70,5.50,-9.90,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52942,52942,2022-10-24,강원,8.0,10.0,1.40,1.00,4.10,112.5,82.80,...,82.70,95.50,6.60,11.05,1.60,0.0,0.0,0.0,0.0,0.0
52958,52958,2022-10-25,강원,6.5,20.0,1.20,0.70,4.00,135.0,87.50,...,84.40,116.80,6.40,13.50,0.80,0.0,0.0,0.0,0.0,0.0
52969,52969,2022-10-26,강원,5.5,60.0,1.10,0.50,3.50,180.0,206.50,...,210.90,193.65,7.90,17.40,0.70,0.0,0.0,0.0,0.0,0.0
52985,52985,2022-10-27,강원,3.0,57.5,1.00,0.50,3.80,202.5,207.10,...,224.60,246.80,9.20,19.40,2.90,0.0,0.0,0.0,0.0,0.0


In [54]:
# 데이터 새로 불러와봤자 어차피 빠진 날짜들이 또 있을거야
# 일단 이전에 전처리 했던 과정을 그대로 따라가도 될 것 같음. 어차피 빠진 날짜는 있을테니까
# 그 다음에 원데이터에서 날짜처리된 열하고 실제 날짜를 생성해서 Outer join시키면 없던 날짜들이 행으로 추가가 될거고, 
# 추가가 된 행에는 기상데이터가 없었으니까 변수별 관측값들이 모두 비어있겠지
# 먼저 시도별로 나눈 다음에, 이 빈 날짜들에 어차피 수치형 변수 밖에 없으니까 한해서 linear interpolation해주면 될거야.
# 먼저 있는 데이터로 이거에 대한 코드 prototype을 만들어놓자
# 데이터 테이블별로 진행하는게 좋겠지...?